	-- AGG KOMUTLARI
SELECT 
	BRANCH AS [SUBE ADI],
	SUM(LINENET) AS [TOPLAM SATIS],
	COUNT(LINENET) AS [SATIS SAYISI],
	MIN(LINENET) AS [MINIMUM SATIS],
	MAX(LINENET) AS [MAKSIMUM SATIS],
	AVG(LINENET) AS [ORTALAMA SATIS]
FROM
	SALES
GROUP BY
	BRANCH
ORDER BY
	BRANCH

	-- GROUP BY VE HAVING KOMUTU
SELECT 
	BRANCH AS [SUBE ADI],
	COUNT(LINENET) AS [SATIS SAYISI]
FROM
	SALES
GROUP BY
	BRANCH
HAVING 
	COUNT(LINENET) > 15000
ORDER BY
	[SATIS SAYISI] DESC

	-- ŞUBELERİN GÜNLÜK SATIŞLARI
SELECT
	DATE_,
	BRANCH,
	SUM(LINENET) AS SATIS
FROM
	SALES
GROUP BY 
	DATE_, BRANCH
ORDER BY
	DATE_ 

	-- KATEGORİLERE GÖRE SATIS TUTARI
SELECT 
	CATEGORY_NAME1, 
	CATEGORY_NAME2,
	SUM(LINENET) AS [SATIS TUTARI]
FROM 
	SALES
WHERE
	CATEGORY_NAME1 IS NOT NULL
GROUP BY 
	CATEGORY_NAME1, 
	CATEGORY_NAME2
HAVING 
	SUM(LINENET) > 30000
ORDER BY
	[SATIS TUTARI] DESC

	-- MUSTERILERIN GITTIGI MAGAZA SAYILARI
SELECT
	CLIENTNAME, 
	COUNT(DISTINCT BRANCH) AS [MAGAZA SAYISI]
FROM
	SALES
WHERE
	CLIENTNAME IS NOT NULL
GROUP BY 
	CLIENTNAME
HAVING 
	COUNT(DISTINCT BRANCH) > 1
ORDER BY
	[MAGAZA SAYISI] DESC

/*
TRUNCATE TABLE BASKET
TRUNCATE TABLE BASKETDETAIL
TRUNCATE TABLE PAYMENT
TRUNCATE TABLE ORDERDETAIL
TRUNCATE TABLE ORDER_
TRUNCATE TABLE INVOICE
TRUNCATE TABLE INVOICEDETAIL
*/
DECLARE @I AS INT=0
WHILE @I<1
BEGIN
DECLARE @USERID AS INT
DECLARE @BASKETID AS INT
DECLARE @ITEMID AS INT
DECLARE @ORDERID AS INT
DECLARE @DATE AS DATETIME

DECLARE @DATE2 AS DATETIME
DECLARE @RAND AS FLOAT
DECLARE @ADDRESID AS INT 

--USER SEÇİLİYOR
SET @RAND =RAND()*10000
SET @USERID =@RAND
SET @RAND =RAND()*365
SET @DATE =DATEADD(DAY,-@RAND,GETDATE())
SET @DATE2=@DATE 
DECLARE @ADDRESSCOUNT AS INT
SELECT @ADDRESSCOUNT=COUNT(*) FROM ADDRES WHERE USERID=@USERID 

SELECT @ADDRESID=ID  FROM (SELECT 
  ROW_NUMBER() OVER(  ORDER BY ID ASC) AS ROWNR,
* FROM ADDRES WHERE USERID=@USERID
) T WHERE ROWNR=ROUND(RAND()*@ADDRESSCOUNT,0)
--BASKET
INSERT INTO BASKET(USERID, CREATEDDATE, STATUS_)
VALUES (@USERID,@DATE,0)
SET @BASKETID=@@IDENTITY
--BASKETDETAIL
DECLARE @ITEMCOUNT AS INT
SET @ITEMCOUNT=RAND()*10
DECLARE @K AS INT=0
DECLARE @AMOUNT AS INT
	WHILE @K<@ITEMCOUNT
	BEGIN
		SET @ITEMID=RAND()*28110
		SET @AMOUNT =1+(RAND()*4)
		SET @DATE=ISNULL((SELECT MAX(DATE_) FROM BASKETDETAIL WHERE BASKETID=@BASKETID),@DATE)
		SET @DATE=DATEADD(SECOND,RAND()*150, @DATE)
		INSERT INTO BASKETDETAIL( BASKETID, DATE_, ITEMID, AMOUNT, PRICE,  TOTAL)
		SELECT   @BASKETID,@DATE,@ITEMID,@AMOUNT,PRICE,PRICE*@AMOUNT FROM ITEM WHERE ID=@ITEMID
		SET @K=@K+1
		UPDATE BASKET SET LASTMODIFIEDDATE=@DATE WHERE ID=@BASKETID
	END
----PAYMENT
DECLARE @TOTALPRICE AS FLOAT
DECLARE @APPROVECODE AS VARCHAR(20)
SET @DATE =DATEADD(SECOND,RAND()*50,@DATE)
SET @APPROVECODE='OK'+CONVERT(VARCHAR,@BASKETID)+CONVERT(VARCHAR,YEAR(@DATE)*365+DAY(@DATE)+DATEPART(SECOND,@DATE))
SELECT @TOTALPRICE=SUM(AMOUNT*PRICE) FROM BASKETDETAIL WHERE BASKETID=@BASKETID
INSERT INTO PAYMENT (BASKETID, TOTALPRICE, PAYMENTTYPE, DATE_, ISOK, APPROVECODE, ERROR_)
VALUES (@BASKETID, @TOTALPRICE, ROUND(RAND()*5,0), @DATE, 1, @APPROVECODE, '')


UPDATE BASKET SET STATUS_=1 WHERE ID=@BASKETID
--ORDER
INSERT INTO ORDER_ ( USERID, ADDRESSID, BASKETID, DATE_, STATUS_)
VALUES (@USERID,@ADDRESID,@BASKETID,@DATE,1)

SET @ORDERID=@@IDENTITY
--ORDERDETAIL
DECLARE @INVOICEID AS INT

INSERT INTO ORDERDETAIL(ORDERID, BASKETDETAILID, ITEMID, AMOUNT, PRICE, TOTALPRICE)
SELECT @ORDERID, ID, ITEMID, AMOUNT, PRICE, AMOUNT*PRICE ASTOTALPRICE FROM BASKETDETAIL WHERE BASKETID=@BASKETID

--INVOICE
DECLARE @INVOICENO AS VARCHAR(100)
SELECT  @INVOICENO=REPLACE(CONVERT(VARCHAR,GETDATE(),104),'.','')+'-'+REPLACE(CONVERT(VARCHAR,GETDATE(),108),':','')
INSERT INTO [dbo].[INVOICE] (ORDERID, INVOICENO, DATE_, CARGOFICHENO, STATUS_)
VALUES (@ORDERID,@INVOICENO,DATEADD(SECOND,(100000+RAND()*250000),@DATE),'CRG'+@INVOICENO,1)


 SET @INVOICEID=@@IDENTITY

INSERT INTO INVOICEDETAIL (INVOICEID, ORDERDETAILID, ITEMID, PRICE, AMOUNT,TOTALPRICE)
SELECT @INVOICEID, ID, ITEMID, PRICE, AMOUNT, PRICE*AMOUNT FROM ORDERDETAIL WHERE ORDERID=@ORDERID
--SELECT @USERID,@DATE ,@ADDRESSCOUNT,@ADDRESID

SET @I=@I+1 
END

SELECT 
	U.USERNAME_ AS [Kullanıcı Adı], 
	U.NAMESURNAME AS [İsim Soyisim],
	A.ADDRESSTEXT AS [Adres], 
	C.CITY AS [Şehir],
	COUNT(*) AS [Sayı]
FROM
	USER_ AS U
JOIN ADDRES AS A ON U.ID=A.USERID
JOIN CITY AS C ON C.ID=A.CITYID
JOIN TOWN AS T ON T.ID=A.TOWNID
JOIN DISTRICT AS D ON D.ID=A.DISTRICTID
--WHERE 
--	U.ID=1
GROUP BY 
	U.USERNAME_, 
	U.NAMESURNAME,
	A.ADDRESSTEXT,
	C.CITY
HAVING 
	COUNT(*) = 1
ORDER BY
	4 DESC,
	[Kullanıcı Adı] ASC